In [1]:
from autograd import jacobian, hessian, grad
import autograd.numpy as np
from autograd.scipy.stats import norm
#from scipy.stats import norm
#from jax import jacobian, hessian, grad
#from jax.scipy.stats import norm
#import jax.numpy as np
#import parametric as par

import numpy as onp
from scipy.optimize import minimize

In [2]:
from numpy.linalg import inv

In [3]:
NUM = np.float32

In [4]:
TINIEST = np.finfo(NUM).tiny

In [7]:
def sf(x, alpha, beta):
    return np.exp(-(x / alpha)**beta)

def ff(x, alpha, beta):
    return 1 - np.exp(-(x / alpha)**beta)

def df(x, alpha, beta):
    return (beta / alpha) * (x / alpha)**(beta-1) * np.exp(-(x / alpha)**beta)

#def sf(x, mu, sigma):
    return 1 - norm.cdf(x, mu, sigma)

#def ff(x, mu, sigma):
    return norm.cdf(x, mu, sigma)

#def df(x, mu, sigma):
    return norm.pdf(x, mu, sigma)

In [8]:
def neg_ll(params, n, c, x):
    like = (n * ff(x, *params) * c * (c - 1.) / 2. +
            n * df(x, *params) * (1. - c**2.) +
            n * sf(x, *params) * c * (c + 1.) / 2.
               )
    like += TINIEST

    return np.array(-np.sum(np.log(like)))

In [9]:
x = np.array([32, 33, 34, 35, 36, 37, 38, 39, 40, 42]).astype(NUM)
n = np.array([10, 23, 48, 80, 63, 65, 47, 33, 14,  6]).astype(NUM)
c = None
if n is None:
    n = np.ones_like(x).astype(NUM)

if c is None:
    c = np.zeros_like(x).astype(NUM)

In [10]:
dR_dthetas = np.vectorize(jacobian(lambda x : sf(x, 5., 7.)))

In [11]:
fun_nll_jax = lambda xx : neg_ll(xx, n, c, x)
fun_nll = lambda xx : onp.array(neg_ll(xx, n, c, x))

In [12]:
fun_nll([50., 2.])

array(6.9518023, dtype=float32)

In [13]:
jac = jacobian(fun_nll_jax)
hess = hessian(fun_nll_jax)

In [14]:
jac(np.array([32., 2.]))

array([-0.19824219, -4.40390861])

In [15]:
x = np.array([16, 34, 53, 75, 93, 120])
c = None
n = None
if n is None:
    n = np.ones_like(x).astype(NUM)

if c is None:
    c = np.zeros_like(x).astype(NUM)

In [16]:
res = minimize(fun_nll, np.array([np.mean(x),  np.std(x)]), method='trust-exact', jac=jac, hess=hess)

In [17]:
res

     fun: array(29.58492162)
    hess: array([[ 0.00414577, -0.03221496],
       [-0.03221496,  2.62502481]])
     jac: array([-1.61529534e-06,  5.17930935e-05])
 message: 'Optimization terminated successfully.'
    nfev: 19
    nhev: 19
     nit: 18
    njev: 17
  status: 0
 success: True
       x: array([73.52581296,  1.93269451])

In [18]:
hess_inv = inv(res.hess)

In [19]:
hess_inv

array([[266.63710784,   3.27223702],
       [  3.27223702,   0.42110649]])